In [1]:
import os
import json
import nltk
import gensim
import numpy as np
from gensim import corpora, models, similarities
import pickle
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from itertools import chain

In [2]:
from keras.models import Sequential
from keras.layers import GRU, RepeatVector
from keras.layers.recurrent import LSTM, SimpleRNN
from sklearn.model_selection import train_test_split

/Users/championlin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
file = open('conversation.json')
data = json.load(file)
cor = data['conversations']
sentenceList=list(chain(*cor))
# sentence = [sentence_to_wordlist(p) for p in  list(chain(*[n for n in cor]))]

In [24]:
model = gensim.models.Word2Vec.load('./doc2vec/toy_data/model.bin')
# model = gensim.models.KeyedVectors.load('./wiki_sg/word2vec.bin', binary=True)

In [14]:
x = []
y = []

In [15]:
len(cor)

13

In [16]:
for i in range(len(cor)):
    for j in range(len(cor[i])):
        if j<len(cor[i])-1:
            x.append(cor[i][j])
            y.append(cor[i][j+1])

In [17]:
x[0]

'Good morning, how are you?'

In [18]:
nltk.word_tokenize(x[0].lower())

['good', 'morning', ',', 'how', 'are', 'you', '?']

In [19]:
x_tok=[]
y_tok=[]
for i in range(len(x)):
    x_tok.append(nltk.word_tokenize(x[i].lower()))
    y_tok.append(nltk.word_tokenize(y[i].lower()))

In [25]:
x_tok

[['good', 'morning', ',', 'how', 'are', 'you', '?'],
 ['i', 'am', 'doing', 'well', ',', 'how', 'about', 'you', '?'],
 ['i', "'m", 'also', 'good', '.'],
 ['that', "'s", 'good', 'to', 'hear', '.'],
 ['hello'],
 ['hi'],
 ['how', 'are', 'you', 'doing', '?'],
 ['i', 'am', 'doing', 'well', '.'],
 ['that', 'is', 'good', 'to', 'hear'],
 ['yes', 'it', 'is', '.'],
 ['can', 'i', 'help', 'you', 'with', 'anything', '?'],
 ['yes', ',', 'i', 'have', 'a', 'question', '.'],
 ['what', 'is', 'your', 'question', '?'],
 ['could', 'i', 'borrow', 'a', 'cup', 'of', 'sugar', '?'],
 ['i', "'m", 'sorry', ',', 'but', 'i', 'do', "n't", 'have', 'any', '.'],
 ['thank', 'you', 'anyway'],
 ['how', 'are', 'you', 'doing', '?'],
 ['i', 'am', 'doing', 'well', ',', 'how', 'about', 'you', '?'],
 ['i', 'am', 'also', 'good', '.'],
 ['have', 'you', 'heard', 'the', 'news', '?'],
 ['what', 'is', 'your', 'favorite', 'book', '?'],
 ['i', 'ca', "n't", 'read', '.'],
 ['so', 'what', "'s", 'your', 'favorite', 'color', '?'],
 ['who', '

In [26]:
sentend=np.ones((300,),dtype=np.float32) 

vec_x=[]
for sent in x_tok:
    sentvec = [model[w] for w in sent if w in model]
    vec_x.append(sentvec)

/Users/championlin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
/Users/championlin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [27]:
vec_y=[]
for sent in y_tok:
    sentvec = [model[w] for w in sent if w in model]
    vec_y.append(sentvec)

/Users/championlin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/championlin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
for tok_sent in vec_x:
    tok_sent[14:]=[]
    tok_sent.append(sentend)
    

for tok_sent in vec_x:
    if len(tok_sent)<15:
        for i in range(15-len(tok_sent)):
            tok_sent.append(sentend)    
            
for tok_sent in vec_y:
    tok_sent[14:]=[]
    tok_sent.append(sentend)
    

for tok_sent in vec_y:
    if len(tok_sent)<15:
        for i in range(15-len(tok_sent)):
            tok_sent.append(sentend)   

In [29]:
vec_x = np.array(vec_x)
vec_y = np.array(vec_y) 

In [30]:
vec_x[0]

array([[ 0.01195055, -0.39919388, -0.32576656, ...,  0.31646454,
        -0.56447613, -0.3012075 ],
       [ 0.23522253,  0.590183  , -0.27571753, ...,  0.08240149,
         0.04993955,  0.06463409],
       [ 0.18240982,  0.3054836 , -0.27625182, ...,  0.19336936,
         0.21038434, -0.0192353 ],
       ...,
       [ 1.        ,  1.        ,  1.        , ...,  1.        ,
         1.        ,  1.        ],
       [ 1.        ,  1.        ,  1.        , ...,  1.        ,
         1.        ,  1.        ],
       [ 1.        ,  1.        ,  1.        , ...,  1.        ,
         1.        ,  1.        ]], dtype=float32)

In [31]:
x_train, x_test, y_train,y_test = train_test_split(vec_x, vec_y, test_size=0.2, random_state=1)

In [32]:
x_train.shape

(68, 15, 300)

In [33]:
model = Sequential()
model.add(LSTM(input_shape=(15, 300), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
model.add(LSTM(input_shape=(15, 300), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
model.add(LSTM(input_shape=(15, 300), return_sequences=False, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
# model.add(GRU(input_shape=(15, 300), units=300, return_sequences=True))
# model.add(GRU(input_shape=(15, 300), units=300, return_sequences=True))
# model.add(SimpleRNN(units=50, input_shape=(15, 50), activation="sigmoid", return_sequences=True))
# model.add(LSTM(input_shape=(15, 300), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
model.add(RepeatVector(15))
model.add(LSTM(input_shape=(15, 300), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
model.add(LSTM(input_shape=(15, 300), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
model.add(LSTM(input_shape=(15, 300), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
model.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy'])

In [34]:
model.fit(x_train, y_train, nb_epoch=500,validation_data=(x_test, y_test))

/Users/championlin/anaconda3/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 68 samples, validate on 18 samples
Epoch 1/500
68/68 [==============================] - 7s 105ms/step - loss: -0.5968 - acc: 0.0029 - val_loss: -0.5680 - val_acc: 0.0593
Epoch 2/500
68/68 [==============================] - 2s 27ms/step - loss: -0.6195 - acc: 0.0275 - val_loss: -0.5807 - val_acc: 0.0889
Epoch 3/500
68/68 [==============================] - 2s 28ms/step - loss: -0.6244 - acc: 0.0667 - val_loss: -0.5877 - val_acc: 0.1333
Epoch 4/500
68/68 [==============================] - 2s 28ms/step - loss: -0.6297 - acc: 0.1147 - val_loss: -0.5936 - val_acc: 0.1444
Epoch 5/500
68/68 [==============================] - 2s 32ms/step - loss: -0.6345 - acc: 0.1382 - val_loss: -0.5982 - val_acc: 0.1444
Epoch 6/500
68/68 [==============================] - 2s 27ms/step - loss: -0.6391 - acc: 0.1382 - val_loss: -0.6009 - val_acc: 0.1444
Epoch 7/500
68/68 [==============================] - 2s 29ms/step - loss: -0.6423 - acc: 0.1225 - val_loss: -0.6027 - val_acc: 0.1259
Epoch 8/500
68/68

68/68 [==============================] - 2s 28ms/step - loss: -0.7108 - acc: 0.1235 - val_loss: -0.6734 - val_acc: 0.1185
Epoch 62/500
68/68 [==============================] - 2s 27ms/step - loss: -0.7106 - acc: 0.1235 - val_loss: -0.6844 - val_acc: 0.1296
Epoch 63/500
68/68 [==============================] - 2s 28ms/step - loss: -0.7132 - acc: 0.1304 - val_loss: -0.6896 - val_acc: 0.1444
Epoch 64/500
68/68 [==============================] - 2s 32ms/step - loss: -0.7108 - acc: 0.1382 - val_loss: -0.6889 - val_acc: 0.1444
Epoch 65/500
68/68 [==============================] - 2s 30ms/step - loss: -0.7125 - acc: 0.1382 - val_loss: -0.6857 - val_acc: 0.1407
Epoch 66/500
68/68 [==============================] - 3s 39ms/step - loss: -0.7140 - acc: 0.1314 - val_loss: -0.6813 - val_acc: 0.1296
Epoch 67/500
68/68 [==============================] - 2s 29ms/step - loss: -0.7133 - acc: 0.1294 - val_loss: -0.6819 - val_acc: 0.1296
Epoch 68/500
68/68 [==============================] - 3s 37ms/step -

68/68 [==============================] - 2s 27ms/step - loss: -0.7149 - acc: 0.1343 - val_loss: -0.6840 - val_acc: 0.1444
Epoch 122/500
68/68 [==============================] - 2s 26ms/step - loss: -0.7161 - acc: 0.1382 - val_loss: -0.6908 - val_acc: 0.1444
Epoch 123/500
68/68 [==============================] - 2s 25ms/step - loss: -0.7154 - acc: 0.1382 - val_loss: -0.6861 - val_acc: 0.1444
Epoch 124/500
68/68 [==============================] - 2s 24ms/step - loss: -0.7166 - acc: 0.1382 - val_loss: -0.6740 - val_acc: 0.1370
Epoch 125/500
68/68 [==============================] - 2s 24ms/step - loss: -0.7120 - acc: 0.1333 - val_loss: -0.6740 - val_acc: 0.1370
Epoch 126/500
68/68 [==============================] - 2s 22ms/step - loss: -0.7138 - acc: 0.1333 - val_loss: -0.6845 - val_acc: 0.1407
Epoch 127/500
68/68 [==============================] - 2s 34ms/step - loss: -0.7165 - acc: 0.1373 - val_loss: -0.6889 - val_acc: 0.1444
Epoch 128/500
68/68 [==============================] - 2s 32ms

68/68 [==============================] - 2s 33ms/step - loss: -0.7186 - acc: 0.1343 - val_loss: -0.6873 - val_acc: 0.1444
Epoch 182/500
68/68 [==============================] - 2s 28ms/step - loss: -0.7202 - acc: 0.1382 - val_loss: -0.6910 - val_acc: 0.1444
Epoch 183/500
68/68 [==============================] - 3s 40ms/step - loss: -0.7172 - acc: 0.1382 - val_loss: -0.6920 - val_acc: 0.1444
Epoch 184/500
68/68 [==============================] - 2s 30ms/step - loss: -0.7183 - acc: 0.1382 - val_loss: -0.6886 - val_acc: 0.1444
Epoch 185/500
68/68 [==============================] - 2s 23ms/step - loss: -0.7198 - acc: 0.1373 - val_loss: -0.6830 - val_acc: 0.1370
Epoch 186/500
68/68 [==============================] - 2s 33ms/step - loss: -0.7190 - acc: 0.1353 - val_loss: -0.6872 - val_acc: 0.1444
Epoch 187/500
68/68 [==============================] - 2s 31ms/step - loss: -0.7206 - acc: 0.1324 - val_loss: -0.6870 - val_acc: 0.1222
Epoch 188/500
68/68 [==============================] - 2s 25ms

68/68 [==============================] - 2s 23ms/step - loss: -0.7205 - acc: 0.1353 - val_loss: -0.6860 - val_acc: 0.1333
Epoch 242/500
68/68 [==============================] - 2s 22ms/step - loss: -0.7203 - acc: 0.1333 - val_loss: -0.6835 - val_acc: 0.0963
Epoch 243/500
68/68 [==============================] - 2s 22ms/step - loss: -0.7203 - acc: 0.1284 - val_loss: -0.6821 - val_acc: 0.0963
Epoch 244/500
68/68 [==============================] - 2s 25ms/step - loss: -0.7207 - acc: 0.1284 - val_loss: -0.6837 - val_acc: 0.0963
Epoch 245/500
68/68 [==============================] - 2s 23ms/step - loss: -0.7208 - acc: 0.1304 - val_loss: -0.6834 - val_acc: 0.0963
Epoch 246/500
68/68 [==============================] - 2s 24ms/step - loss: -0.7212 - acc: 0.1304 - val_loss: -0.6813 - val_acc: 0.0963
Epoch 247/500
68/68 [==============================] - 2s 32ms/step - loss: -0.7208 - acc: 0.1324 - val_loss: -0.6812 - val_acc: 0.1074
Epoch 248/500
68/68 [==============================] - 2s 32ms

68/68 [==============================] - 2s 24ms/step - loss: -0.7196 - acc: 0.4373 - val_loss: -0.6807 - val_acc: 0.3852
Epoch 302/500
68/68 [==============================] - 2s 24ms/step - loss: -0.7199 - acc: 0.4255 - val_loss: -0.6843 - val_acc: 0.0926
Epoch 303/500
68/68 [==============================] - 2s 24ms/step - loss: -0.7218 - acc: 0.1304 - val_loss: -0.6864 - val_acc: 0.0926
Epoch 304/500
68/68 [==============================] - 2s 23ms/step - loss: -0.7209 - acc: 0.1304 - val_loss: -0.6864 - val_acc: 0.0963
Epoch 305/500
68/68 [==============================] - 2s 24ms/step - loss: -0.7208 - acc: 0.1284 - val_loss: -0.6857 - val_acc: 0.0963
Epoch 306/500
68/68 [==============================] - 2s 26ms/step - loss: -0.7213 - acc: 0.1294 - val_loss: -0.6839 - val_acc: 0.1037
Epoch 307/500
68/68 [==============================] - 2s 26ms/step - loss: -0.7216 - acc: 0.1265 - val_loss: -0.6845 - val_acc: 0.1074
Epoch 308/500
68/68 [==============================] - 2s 27ms

68/68 [==============================] - 2s 24ms/step - loss: -0.7196 - acc: 0.1353 - val_loss: -0.6866 - val_acc: 0.1259
Epoch 362/500
68/68 [==============================] - 2s 24ms/step - loss: -0.7185 - acc: 0.1363 - val_loss: -0.6867 - val_acc: 0.1333
Epoch 363/500
68/68 [==============================] - 2s 23ms/step - loss: -0.7199 - acc: 0.1314 - val_loss: -0.6804 - val_acc: 0.1037
Epoch 364/500
68/68 [==============================] - 2s 27ms/step - loss: -0.7213 - acc: 0.1265 - val_loss: -0.6792 - val_acc: 0.1037
Epoch 365/500
68/68 [==============================] - 2s 23ms/step - loss: -0.7211 - acc: 0.1245 - val_loss: -0.6817 - val_acc: 0.1074
Epoch 366/500
68/68 [==============================] - 2s 22ms/step - loss: -0.7220 - acc: 0.1265 - val_loss: -0.6830 - val_acc: 0.1407
Epoch 367/500
68/68 [==============================] - 2s 22ms/step - loss: -0.7221 - acc: 0.1373 - val_loss: -0.6839 - val_acc: 0.1444
Epoch 368/500
68/68 [==============================] - 2s 24ms

68/68 [==============================] - 2s 23ms/step - loss: -0.7222 - acc: 0.1265 - val_loss: -0.6767 - val_acc: 0.0963
Epoch 422/500
68/68 [==============================] - 1s 22ms/step - loss: -0.7233 - acc: 0.1265 - val_loss: -0.6824 - val_acc: 0.0963
Epoch 423/500
68/68 [==============================] - 1s 22ms/step - loss: -0.7238 - acc: 0.1304 - val_loss: -0.6850 - val_acc: 0.0963
Epoch 424/500
68/68 [==============================] - 2s 22ms/step - loss: -0.7242 - acc: 0.1284 - val_loss: -0.6840 - val_acc: 0.1185
Epoch 425/500
68/68 [==============================] - 2s 22ms/step - loss: -0.7240 - acc: 0.1333 - val_loss: -0.6831 - val_acc: 0.1296
Epoch 426/500
68/68 [==============================] - 2s 22ms/step - loss: -0.7246 - acc: 0.1314 - val_loss: -0.6831 - val_acc: 0.1296
Epoch 427/500
68/68 [==============================] - 2s 22ms/step - loss: -0.7247 - acc: 0.1353 - val_loss: -0.6838 - val_acc: 0.1481
Epoch 428/500
68/68 [==============================] - 2s 23ms

68/68 [==============================] - 2s 30ms/step - loss: -0.7350 - acc: 0.1363 - val_loss: -0.6713 - val_acc: 0.1407
Epoch 482/500
68/68 [==============================] - 2s 34ms/step - loss: -0.7347 - acc: 0.1343 - val_loss: -0.6773 - val_acc: 0.1407
Epoch 483/500
68/68 [==============================] - 2s 33ms/step - loss: -0.7366 - acc: 0.1353 - val_loss: -0.6590 - val_acc: 0.1148
Epoch 484/500
68/68 [==============================] - 2s 33ms/step - loss: -0.7273 - acc: 0.1275 - val_loss: -0.6758 - val_acc: 0.1074
Epoch 485/500
68/68 [==============================] - 2s 27ms/step - loss: -0.7332 - acc: 0.1284 - val_loss: -0.6777 - val_acc: 0.1074
Epoch 486/500
68/68 [==============================] - 2s 25ms/step - loss: -0.7329 - acc: 0.1304 - val_loss: -0.6610 - val_acc: 0.0926
Epoch 487/500
68/68 [==============================] - 2s 23ms/step - loss: -0.7300 - acc: 0.1225 - val_loss: -0.6762 - val_acc: 0.1074
Epoch 488/500
68/68 [==============================] - 2s 23ms

In [35]:
model.save('LSTM500.h5')